In [ ]:
!pip install pickle5

     |████████████████████████████████| 256 kB 4.2 MB/s 


In [ ]:
import gensim
import pickle5
from gensim import corpora, models
import pprint

In [ ]:
with open('ukraine_processed.pickle', 'rb') as f:
  data = pickle5.load(f)

In [ ]:
len(data)

9979

In [ ]:
print(data[:5])

[['go', 'start', 'isnt', 'rhetor', 'west', 'increas', 'tension', 'amp', 'one', 'misplac', 'phrase', 'could', 'trigger', 'conflict', 'send', '🤡'], ['hm', 'may', 'want', 'check', 'biden', 'famili', 'tie', 'amp', 'china', '’', 'great', 'stori', 'way…jo', '’', 'big', 'guy', 'storylin'], ['bring', 'back', 'militari', 'infrastructur', 'allianc', 'statu', 'quo', 'russianato', 'treati', 'sign'], ['presid', 'putin', 'russian', 'concern', 'basic', 'ignoredw', '’', 'see', 'adequ', 'respons', 'key', 'concern', 'nonexpans', 'nato', 'refus', 'deploy', 'offens', 'weapon', 'next', 'russian', 'border'], ['bombshel', 'key', 'evid', 'undermin', 'impeach', 'narr', 'withheld', 'trump', 'defens']]


Vamos a crear un diccionario con la información procesada que contenga el número de veces que una palabra aparezca en los documentos.

In [ ]:
dictionary = gensim.corpora.Dictionary(data)

In [ ]:
count = 0
for k, v in dictionary.iteritems():
  print(k, v)
  count += 1
  if count > 10:
    break

0 amp
1 conflict
2 could
3 go
4 increas
5 isnt
6 misplac
7 one
8 phrase
9 rhetor
10 send


In [ ]:
len(dictionary)

1451

Filtrar las palabras que aparezcan:


*   en menos de 15 tweets
*   más de 1 tweet
* conservar los 100000 tokens más frecuentes



In [ ]:
dictionary.filter_extremes(no_below=15, no_above=1, keep_n=100000)


Para cada documento creamos un diccionario informando cuántas
palabras y cuántas veces aparecen. Se guarda esto en `bow_corpus`.



In [ ]:
bow_corpus = [dictionary.doc2bow(doc) for doc in data]

In [ ]:
bow_corpus[1]

[(0, 1),
 (13, 1),
 (14, 1),
 (15, 1),
 (16, 1),
 (17, 1),
 (18, 1),
 (19, 1),
 (20, 1),
 (21, 1),
 (22, 1),
 (23, 1),
 (24, 2)]

In [ ]:
bow_doc_1 = bow_corpus[0]

for i in range(len(bow_doc_1)):
  print("Word {} (\"{}\") appears {} time.".format(bow_doc_1[i][0], 
                                               dictionary[bow_doc_1[i][0]], 
                                               bow_doc_1[i][1])) 

Word 0 ("amp") appears 1 time.
Word 1 ("conflict") appears 1 time.
Word 2 ("could") appears 1 time.
Word 3 ("go") appears 1 time.
Word 4 ("increas") appears 1 time.
Word 5 ("isnt") appears 1 time.
Word 6 ("one") appears 1 time.
Word 7 ("rhetor") appears 1 time.
Word 8 ("send") appears 1 time.
Word 9 ("start") appears 1 time.
Word 10 ("tension") appears 1 time.
Word 11 ("trigger") appears 1 time.
Word 12 ("west") appears 1 time.


In [ ]:
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]

In [ ]:
for doc in corpus_tfidf:
  pprint.pprint(doc)
  break

[(0, 0.1850349951440687),
 (1, 0.26601864479358556),
 (2, 0.2510749876663248),
 (3, 0.19271678623717745),
 (4, 0.3386186475170189),
 (5, 0.32162597538577486),
 (6, 0.2311765284770271),
 (7, 0.37790632989521894),
 (8, 0.23859156812216317),
 (9, 0.26909259350176234),
 (10, 0.21545773953039843),
 (11, 0.38903387847731846),
 (12, 0.23097603273752457)]


In [ ]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)

In [ ]:
for idx, topic in lda_model_tfidf.print_topics(-1):
  print('Topic: {} Word: {}'.format(idx, topic))

In [ ]:
for index, score in sorted(lda_model_tfidf[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
  print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))



---



Vamos a realizar una máquina de vectores de soporte.